In [1]:
import pymongo
client = pymongo.MongoClient(### place your mangodb url###)
db = client.Task
records = db.aathar
def menu():
    
  print("Welcome \n"
  "1. Register \n"
  "2. update \n"
  "3. Download Pdf")
  cursor = int(input("Enter a number to get started"))
  
  if cursor == 1:
    reg()
  elif cursor == 2:
    update()
  elif cursor == 3:
    pdf()
  else:
    print("Enter a valid number")


def reg():
  name=input("Enter Name")

# image to binary
  from PIL import Image
  import io
  profile_pic=input("Enter the path of file: ")
  im = Image.open(profile_pic)
  image_prof = io.BytesIO()
  im.save(image_prof, format='PNG')


  dob=input("Enter DOB in DD/MM/YYYY format: ")


# Getting address
  j=0
  while j <1:
      add=list(input("ADDRESS in format No,Street,City,State,Pincode").split(","))
      
      if len(add)==5:
          No=add[0]
          Street=add[1]
          City=add[2]
          State=add[3]
          Pincode=add[4]
          j+=1
      else:
          print("Enter a Valid Address")
      

  gen=input("Enter your gender: ")

# Getting Qualification
  i=0
  while i<1:
      education_qualification=input("Enter your Highest Qualification NA/10th/12th/Bacelores/Master/Doctrate :")
      education_list=['NA','10th','12th','Bacelores','Master','Doctrate']
      if (education_qualification  in education_list) and (education_qualification != education_list[0]):
        stream = input("Enter Your Stream :")
        i+=1
      elif len(education_qualification) == 2 and education_qualification.upper() == "NA":
          education_qualification='NA'
          stream = "NA"
          i+=1
              
      else:
          print("Enter Valid Education Qualification")

  emailid=input("Enter your Email Address :")

#generating 12-igit Aadhar Number
  import random
  aadhar = (random.randint(100000000000,999999999999))

  from datetime import timedelta, date
  Date_req = date.today() + timedelta(days=730)
  
  
# qr generating
  import qrcode
  a={
      "Aathar": aadhar,
      "Name":name,
     "Date of birth":dob,
     "Address":{"No":No,"Street":Street,"City":City,"State":State,"pincode":Pincode},
     "Gender":gen,
     "Education Qualification":education_qualification,
     "Stream":stream,
     "Email":emailid,
     "Reneval Date": Date_req
     }
  img=qrcode.make(a)
  type(img)
  img.save(f"{name}.jpg")
  im1 = Image.open(f"{name}.jpg")
  image_bytes = io.BytesIO()
  im1.save(image_bytes, format='png')

# Saving part
  a={
      "_id":aadhar,
    "Name":name,
    "image":image_prof.getvalue(),
    "Date of birth":dob,
    "Address":{"No":No,"Street":Street,"City":City,"State":State,"pincode":Pincode},
    "Gender":gen,
    "Education Qualification":education_qualification,
    "Stream":stream,
    "Email":emailid,
    "QR Code":image_bytes.getvalue(),
     }
  records.insert_one(a)

  print("Sucessfully Registered \n"
  f"Your Aathar Number : {aadhar} \n"
  "Here is your Qr Code")
  print(f"Your reneval date is: {Date_req}")
  display(img)


# update
def update():
  from PIL import Image
  import io

  user_aathar = int(input("Enter your aadhar number : "))
  Query = {"_id" : user_aathar}
  print("you can only edit: \n"
  "1.Name\n"
  "2.Profile_pic \n"
  "3.Address \n"
  "4.Educational_qualifivation \n"
  "5.Stream \n"
  "6.Email"
  "7.Profile pic")
  edit = int(input("Enter what you want to edit"))
  if edit == 1:
    n_name = input("Enter your Name")
    new_query = {"$set":{"Name":n_name}}
    records.update_one(Query,new_query)

  elif edit == 2:
    profile_pic=input("Enter the path of file: ")
    im = Image.open(profile_pic)
    image_bytes = io.BytesIO()
    im.save(image_bytes, format='PNG')
    new_query = {"$set":{"image":image_bytes.getvalue()}}
    records.update_one(Query,new_query)

  elif edit == 3:
    j=0
    while j <1:
      add=list(input("ADDRESS in format No,Street,City,State,Pincode").split(","))
      
      if len(add)==5:
          No=add[0]
          Street=add[1]
          City=add[2]
          State=add[3]
          Pincode=add[4]
          j+=1
      else:
          print("Enter a Valid Address : ")
    new_query = {"$set":{"Address":{"No":No,"Street":Street,"City":City,"State":State,"pincode":Pincode}}}
    records.update_one(Query,new_query)

  elif edit ==4:
    n_eq = input("Enter your Educational_qualifivation : ")
    new_query = {"$set":{"Education Qualification":n_eq}}
    records.update_one(Query,new_query)

  elif edit ==5:
    n_stream = input("Enter your Stream : ")
    new_query = {"$set":{"Stream":n_stream}}
    records.update_one(Query,new_query)

  elif edit ==6:
    n_email = input("Enter your Email : ")
    new_query = {"$set":{"Email":n_email}}
    records.update_one(Query,new_query)

  elif edit ==7:
    profile_pic=input("Enter the path of file : ")
    im = Image.open(profile_pic)
    image_bytes = io.BytesIO()
    im.save(image_bytes, format='PNG')   
    new_query = {"$set":{"image":image_bytes.getvalue()}}
    records.update_one(Query,new_query)

  else:
    print("Enter a valid number : ")


def pdf():
   
  from fpdf import FPDF
  
  aadhar = int(input("Enter your aadhar : "))
  pdf = FPDF()
  pdf.add_page()
  pdf.set_font("Arial", "B" , size = 25)
  pdf.cell(200, 10, txt = "Aadhar Card",ln = 1, align = 'C')

  pdf.set_font("Arial", size = 20)
  pdf.cell(200, 10, txt = (f"Aadhar Number : {aadhar}"), ln = 2, align = 'C')
  
  pdf.set_font("Arial", size = 15)
  a = records.find_one({"_id":{"$eq":aadhar}}, {"Name":1})
  for key,value in a.items():
    b =(key ,"=", value)
  pdf.cell(200, 10, txt = (f"{key} : {value}"), ln = 3, align = 'L')

  a = records.find_one({"_id":{"$eq":aadhar}}, {"Date of birth":1})
  for key,value in a.items():
    b =(key ,"=", value)
  pdf.cell(200, 10, txt = (f"{key} : {value}"), ln = 4, align = 'L')
  
  
  a = records.find_one({"_id":{"$eq":aadhar}}, {"Address":1})
  for key,value in a.items():
    b =(key ,"=", value)
  pdf.cell(200, 10, txt = (f"{key}"), ln = 5, align = 'L')
  pdf.set_font("Arial", size = 10)
  pdf.cell(300, 10, txt = (f"{value}"), ln = 6, align = 'L')
  
  
  pdf.set_font("Arial", size = 15)
  a = records.find_one({"_id":{"$eq":aadhar}}, {"Gender":1})
  for key,value in a.items():
    b =(key ,"=", value)
  pdf.cell(200, 10, txt = (f"{key} : {value}"), ln = 7, align = 'L') 

  a = records.find_one({"_id":{"$eq":aadhar}}, {"Education Qualification":1})
  for key,value in a.items():
    b =(key ,"=", value)
  pdf.cell(200, 10, txt = (f"{key} : {value}"), ln = 8, align = 'L') 

  a = records.find_one({"_id":{"$eq":aadhar}}, {"Stream":1})
  for key,value in a.items():
    b =(key ,"=", value)
  pdf.cell(200, 10, txt = (f"{key} : {value}"), ln = 9, align = 'L') 

  a = records.find_one({"_id":{"$eq":aadhar}}, {"Email":1})
  for key,value in a.items():
    b =(key ,"=", value)
  pdf.cell(200, 10, txt = (f"{key} : {value}"), ln = 10, align = 'L')

  

  a = records.find_one({"_id":{"$eq":aadhar}}, {"Name":1})
  for key,value in a.items():
   b =(key , value)
  pdf.output(f"{value}.pdf")


menu()

SyntaxError: ignored